#### Read data

In [1]:
import sframe
import numpy as np
loans = sframe.SFrame('data1/')

[INFO] sframe.cython.cy_server: SFrame v2.1 started. Logging C:\Users\user\AppData\Local\Temp\sframe_server_1502694621.log.0


#### Preprocessing

In [3]:
loans['safe_loans'] = loans['bad_loans'].apply(lambda x : +1 if x==0 else -1)
loans = loans.remove_column('bad_loans')
features = ['grade',              # grade of the loan
            'term',               # the term of the loan
            'home_ownership',     # home_ownership status: own, mortgage or rent
            'emp_length',         # number of years of employment
           ]
target = 'safe_loans'
loans = loans[features + [target]]

In [4]:
loans

grade,term,home_ownership,emp_length,safe_loans
B,36 months,RENT,10+ years,1
C,60 months,RENT,< 1 year,-1
C,36 months,RENT,10+ years,1
C,36 months,RENT,10+ years,1
A,36 months,RENT,3 years,1
E,36 months,RENT,9 years,1
F,60 months,OWN,4 years,-1
B,60 months,RENT,< 1 year,-1
C,60 months,OWN,5 years,1
B,36 months,OWN,10+ years,1


#### Balance Class

In [5]:
safe_loans_raw = loans[loans[target] == 1]
risky_loans_raw = loans[loans[target] == -1]

# Since there are less risky loans than safe loans, find the ratio of the sizes
# and use that percentage to undersample the safe loans.
percentage = len(risky_loans_raw)/float(len(safe_loans_raw))
safe_loans = safe_loans_raw.sample(percentage, seed = 1)
risky_loans = risky_loans_raw
loans_data = risky_loans.append(safe_loans)

print "Percentage of safe loans                 :", len(safe_loans) / float(len(loans_data))
print "Percentage of risky loans                :", len(risky_loans) / float(len(loans_data))
print "Total number of loans in our new dataset :", len(loans_data)

Percentage of safe loans                 : 0.502236174422
Percentage of risky loans                : 0.497763825578
Total number of loans in our new dataset : 46508


#### One hot encoding

In [6]:
loans_data = risky_loans.append(safe_loans)
for feature in features:
    loans_data_one_hot_encoded = loans_data[feature].apply(lambda x: {x: 1})    
    loans_data_unpacked = loans_data_one_hot_encoded.unpack(column_name_prefix=feature)
    
    # Change None's to 0's
    for column in loans_data_unpacked.column_names():
        loans_data_unpacked[column] = loans_data_unpacked[column].fillna(0)

    loans_data.remove_column(feature)
    loans_data.add_columns(loans_data_unpacked)

In [7]:
features = loans_data.column_names()
features.remove('safe_loans')  # Remove the response variable
features

['grade.A',
 'grade.B',
 'grade.C',
 'grade.D',
 'grade.E',
 'grade.F',
 'grade.G',
 'term. 36 months',
 'term. 60 months',
 'home_ownership.MORTGAGE',
 'home_ownership.OTHER',
 'home_ownership.OWN',
 'home_ownership.RENT',
 'emp_length.1 year',
 'emp_length.10+ years',
 'emp_length.2 years',
 'emp_length.3 years',
 'emp_length.4 years',
 'emp_length.5 years',
 'emp_length.6 years',
 'emp_length.7 years',
 'emp_length.8 years',
 'emp_length.9 years',
 'emp_length.< 1 year',
 'emp_length.n/a']

In [8]:
print "Number of features (after binarizing categorical variables) = %s" % len(features)

Number of features (after binarizing categorical variables) = 25


#### Train test split

In [9]:
train_data, test_data = loans_data.random_split(.8, seed=1)

#### Function to calculate number of mistake for a node

In [11]:
a = sframe.SArray([1,2,2,2,1])
a.

In [15]:
def intermediate_node_num_mistakes(labels_in_node):
    if len(labels_in_node) == 0:
        return 0
    number_of_1 = 0
    number_of_not_1 = 0
    for element in labels_in_node:
        if element == 1:
            number_of_1 = number_of_1 + 1
        else:
            number_of_not_1 = number_of_not_1 + 1
    return min(number_of_1,number_of_not_1)

In [16]:
# Test case 1
example_labels = sframe.SArray([-1, -1, 1, 1, 1])
if intermediate_node_num_mistakes(example_labels) == 2:
    print 'Test passed!'
else:
    print 'Test 1 failed... try again!'

# Test case 2
example_labels = sframe.SArray([-1, -1, 1, 1, 1, 1, 1])
if intermediate_node_num_mistakes(example_labels) == 2:
    print 'Test passed!'
else:
    print 'Test 2 failed... try again!'
    
# Test case 3
example_labels = sframe.SArray([-1, -1, -1, -1, -1, 1, 1])
if intermediate_node_num_mistakes(example_labels) == 2:
    print 'Test passed!'
else:
    print 'Test 3 failed... try again!'

Test passed!
Test passed!
Test passed!


#### Function to choose the best feature to split

In [39]:
def best_splitting_feature(data,features,target):
    best_feature = None
    best_error = 10
    num_data_points = float(len(data))
    
    for feature in features:
        left_split = data[data[feature] == 0]
        right_split = data[data[feature] != 0]
        left_mistake = intermediate_node_num_mistakes(left_split[target])
        right_mistake = intermediate_node_num_mistakes(right_split[target])
        error = (left_mistake + right_mistake)/num_data_points
        if error < best_error:
            best_error = error
            best_feature = feature
    return best_feature

In [40]:
if best_splitting_feature(train_data, features, 'safe_loans') == 'term. 36 months':
    print 'Test passed!'
else:
    print 'Test failed... try again!'

Test passed!


#### Build the tree

In [43]:
def create_leaf(target_values):
    
    leaf = {'splitting_feature' : None,
            'left' : None,
            'right' : None,
            'is_leaf': True   }
    num_ones = len(target_values[target_values == +1])
    num_minus_ones = len(target_values[target_values == -1])
    if num_ones > num_minus_ones:
        leaf['prediction'] =  1 
    else:
        leaf['prediction'] =  -1
    return leaf

#### Function to create model

In [41]:
def decision_tree_create(data,features,target,current_depth=0,max_depth=10):
    remaining_features = features[:]
    target_values = data[target]
    print "--------------------------------------------------------------------"
    print "Subtree, depth = %s (%s data points)." % (current_depth, len(target_values))
    
    if intermediate_node_num_mistakes(target_values) == 0:
        print "Reach a leaf node"
        return create_leaf(target_values)
    
    if remaining_features == []:
        print "No more feature to split"
        return create_leaf(target_values)
    
    if current_depth >= max_depth:
        print "Reach maximum depth"
        return create_leaf(target_values)
    
    splitting_feature = best_splitting_feature(data,remaining_features,target)
    left_split = data[data[splitting_feature] == 0]
    right_split = data[data[splitting_feature] != 0]
    remaining_features.remove(splitting_feature)
    print "Split on feature %s. (%s, %s)" % (splitting_feature, len(left_split), len(right_split))
 
    if len(left_split) == len(data):
        print "Creating leaf node."
        return create_leaf(left_split[target])
    if len(right_split) == len(data):
        print "Creating leaf node."
        return create_leaf(right_split[target])
    
    left_tree = decision_tree_create(left_split,remaining_features,target,current_depth + 1, max_depth)
    right_tree = decision_tree_create(right_split,remaining_features,target,current_depth + 1, max_depth)
    return {'is_leaf'          : False, 
            'prediction'       : None,
            'splitting_feature': splitting_feature,
            'left'             : left_tree, 
            'right'            : right_tree}

#### Function to count nodes

In [28]:
def count_nodes(tree):
    if tree['is_leaf']:
        return 1
    return 1 + count_nodes(tree['left']) + count_nodes(tree['right'])

#### Test model

In [44]:
small_data_decision_tree = decision_tree_create(train_data, features, 'safe_loans', max_depth = 3)
if count_nodes(small_data_decision_tree) == 13:
    print 'Test passed!'
else:
    print 'Test failed... try again!'
    print 'Number of nodes found                :', count_nodes(small_data_decision_tree)
    print 'Number of nodes that should be there : 13' 

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term. 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature grade.A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Split on feature grade.B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
Reach maximum depth
--------------------------------------------------------------------
Subtree, depth = 3 (1048 data points).
Reach maximum depth
--------------------------------------------------------------------
Subtree, depth = 2 (101 data points).
Split on feature emp_length.n/a. (96, 5)
--------------------------------------------------------------------
Subtree, depth = 3 (96 data points).
Reach maximum depth
--------------------

#### Build the tree

In [50]:
my_decision_tree = decision_tree_create(train_data,features, 'safe_loans', max_depth = 6)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term. 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature grade.A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Split on feature grade.B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
Split on feature grade.C. (5884, 2190)
--------------------------------------------------------------------
Subtree, depth = 4 (5884 data points).
Split on feature grade.D. (3826, 2058)
--------------------------------------------------------------------
Subtree, depth = 5 (3826 data points).
Split on feature grade.E. (1693, 2133)
--------------------------------------------------------------------
Subtree, depth = 6 (1693 data points).
R

#### Function to classify an instance

In [47]:
def classify(tree,x,annotate = False):
    if tree['is_leaf']:
        if annotate:
            print "At leaf, predicting %s" % tree['prediction']
        return tree["prediction"]
    else:
        split_feature_value = x[tree['splitting_feature']]
        if annotate:
            print "Split on %s = %s" % (tree['splitting_feature'], split_feature_value)
        if split_feature_value == 0:
            return classify(tree['left'], x, annotate)
        else:
            return classify(tree['right'],x,annotate)

#### Start prediction

In [48]:
test_data[0]

{'emp_length.1 year': 0L,
 'emp_length.10+ years': 0L,
 'emp_length.2 years': 1L,
 'emp_length.3 years': 0L,
 'emp_length.4 years': 0L,
 'emp_length.5 years': 0L,
 'emp_length.6 years': 0L,
 'emp_length.7 years': 0L,
 'emp_length.8 years': 0L,
 'emp_length.9 years': 0L,
 'emp_length.< 1 year': 0L,
 'emp_length.n/a': 0L,
 'grade.A': 0L,
 'grade.B': 0L,
 'grade.C': 0L,
 'grade.D': 1L,
 'grade.E': 0L,
 'grade.F': 0L,
 'grade.G': 0L,
 'home_ownership.MORTGAGE': 0L,
 'home_ownership.OTHER': 0L,
 'home_ownership.OWN': 0L,
 'home_ownership.RENT': 1L,
 'safe_loans': -1L,
 'term. 36 months': 0L,
 'term. 60 months': 1L}

In [51]:
print 'Predicted class: %s ' % classify(my_decision_tree, test_data[0])

Predicted class: -1 


#### Evaluate classification erro

In [56]:
def evaluate_classification_error(tree,data,target):
    prediction = data.apply(lambda x: classify(tree,x))
    mistakes = 0
    for i in range(len(prediction)):
        if prediction[i] != data[target][i]:
            mistakes = mistakes + 1
    return mistakes / float(len(prediction))

In [57]:
evaluate_classification_error(my_decision_tree,test_data,target)

0.3837785437311504

#### Print decision map

In [59]:
def print_stump(tree, name = 'root'):
    split_name = tree['splitting_feature'] # split_name is something like 'term. 36 months'
    if split_name is None:
        print "(leaf, label: %s)" % tree['prediction']
        return None
    split_feature, split_value = split_name.split('.')
    print '                       %s' % name
    print '         |---------------|----------------|'
    print '         |                                |'
    print '         |                                |'
    print '         |                                |'
    print '  [{0} == 0]               [{0} == 1]    '.format(split_name)
    print '         |                                |'
    print '         |                                |'
    print '         |                                |'
    print '    (%s)                         (%s)' \
        % (('leaf, label: ' + str(tree['left']['prediction']) if tree['left']['is_leaf'] else 'subtree'),
           ('leaf, label: ' + str(tree['right']['prediction']) if tree['right']['is_leaf'] else 'subtree'))

In [60]:
print_stump(my_decision_tree)

                       root
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [term. 36 months == 0]               [term. 36 months == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (subtree)


In [61]:
print_stump(my_decision_tree['left'], my_decision_tree['splitting_feature'])

                       term. 36 months
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [grade.A == 0]               [grade.A == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (subtree)


In [62]:
print_stump(my_decision_tree['left']['left'], my_decision_tree['left']['splitting_feature'])

                       grade.A
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [grade.B == 0]               [grade.B == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (subtree)
